# Libraries

In [ ]:
import tqdm
import spacy
import numpy as np
import regex as re
import pandas as pd
import seaborn as sns

import pyLDAvis
import pyLDAvis.gensim_models as gensim_models
import pyLDAvis.gensim_models as gensimvis

import gensim
# from gensim.models import wrappers
# from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.corpora import dictionary
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)


# Think these can be removed
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD

# Importing -- Fixing columns

In [3]:
full = pd.read_csv('Data/Data-Cleaned/238k-Uncleaned')

In [4]:
# Renaming tweets column, dropping unnamed column, making tweets strings

full['tweets'] = full['0']
full.reset_index(inplace = True)
full = full.drop(columns = ['0', 'Unnamed: 0', 'index'])
# eh
full.drop_duplicates(inplace = True, ignore_index = True)

# Preprocessing ---- will need to test different preprocessing later

In [7]:
# Function for Gensim simple preprocessor --- 
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        
# STOPWORDS ==========================================
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words]for doc in texts]
# Stop Words list -- Can be Edited
stop_words = stopwords.words('english')
stop_words.extend(['tesla', 'c,', 'x', 't', 'p', 'amp', 'car'])
wnl = WordNetLemmatizer()

# BIGRAMS===============================================
def make_bigrams(texts):
    return bigram_mod[texts]

# TRIGRAMS =============================================
def make_trigrams(texts):
    return trigram_mod[bigram_mod[texts]]

# LEMMATIZING ==========================================
def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None    
def lemmatize_texts(tweet):
     for i in tweet:
        word = (map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
        word_lem = " ".join([wnl.lemmatize(x[0], x[1]) for x in word if x[1] is not None])
        return word_lem

In [8]:
# remove punctuation and lowercase all
full['preprep'] = full['tweets'].map(lambda x: re.sub('[,\.!?]', '', x.lower()))

#  Removing @ handles, links-- strip whitespace breaks and tabs
full['preprep'] = full['preprep'].map(lambda x: re.sub(r"@\w+|http\S+", "", x).strip().replace("\r", "").replace("\n", "").replace("\t", ""))

In [9]:
# full

In [10]:
# Creates a list of all tweets in full.preprep
data = full.preprep.values.tolist()

# Uses gensims simple preprocessor on all tweets in list
# Ouputs list of lists of tokenized tweets
data_words = list(sent_to_words(data))

In [11]:
# Bi / TriGrams
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=75) # higher threshold fewer phrases.
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram = gensim.models.Phrases(bigram[data_words], threshold=60)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# lemmatize
data_lemmatized = [lemmatize_texts(tweet) for tweet in data_words_bigrams]

In [13]:
print(len(data_words), len(data_lemmatized))

126510 126510


In [14]:
# Getting indices of all nones
indices_of_nones = [(i) for i,x  in enumerate(data_lemmatized) if x == None]
len(indices_of_nones)

330

In [15]:
# Dropping None values using indices aquired above
lem_drop_None = [(i, x) for (i, x) in enumerate(data_lemmatized) if i not in indices_of_nones]
len(lem_drop_None)

126180

In [16]:
# Getting indices of all short tweets
indices_short_tweets = [i for (i,x) in lem_drop_None if (len(x.split(' ')) <= 5)]
len(indices_short_tweets)

33324

In [17]:
lem_noShort_noNone = [(i, x) for (i, x) in lem_drop_None if i not in indices_short_tweets]
len(lem_noShort_noNone)

92856

In [18]:
lem_noShort_noNone[8654]

(12588,
 'stock price_target slash wedbush_ives china epic_disaster twitter circus_show teslainc danives wedbush circusshow fintwit')

In [19]:
full.preprep.iloc[12588]

"tesla stock price target slashed by wedbush's ives on china's 'epic disaster' twitter 'circus show' #teslainc #danives #wedbush #circusshow #fintwit"

## Find missing values after making min tweet length 4 words maybe =============================

In [20]:
lem_noNone = [x.split() for (i, x) in lem_noShort_noNone]

In [21]:
# lem_noNone

In [22]:
id2word = corpora.Dictionary(lem_noNone)
print(len(id2word)) 

57412


In [23]:
# This can be Tuned --- and probably should be
id2word.filter_extremes(no_below=6, no_above=.95)
print(len(id2word))

12387


In [24]:
corpus = [id2word.doc2bow(d) for d in lem_noNone]

In [25]:
# Instantiating a Base LDA model
base_model = LdaMulticore(corpus=corpus,
                          num_topics=5,
                          id2word=id2word,
                          workers=7,
                          passes=5,
                          chunksize=2500,
                          random_state = 42)
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

------ Topic 1 ------
musk elon twitter say car ev buy electric crash get

------ Topic 2 ------
get people know think say door company find see elon

------ Topic 3 ------
car make ev go year battery model get electric company

------ Topic 4 ------
stock musk elon twitter buy go price tsla share market

------ Topic 5 ------
car electric get buy drive people want make know think



In [26]:
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=lem_noNone, 
                                   dictionary=id2word, coherence='c_v')

coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base, '\n\nPerplexity: ', base_perplexity)


Coherence Score:  0.291523709329812 

Perplexity:  -7.661196289201235


In [27]:
pyLDAvis.enable_notebook()
gensimvis.prepare(base_model, corpus, id2word)

/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.150928  0.039703       1        1  24.279347
2      0.098684 -0.026241       2        1  23.036273
1      0.042776  0.133675       3        1  19.897816
4      0.091029 -0.052940       4        1  19.029321
0     -0.081562 -0.094196       5        1  13.757243, topic_info=         Term          Freq         Total Category  logprob  loglift
128     stock  12936.000000  12936.000000  Default  30.0000  30.0000
99   electric  11030.000000  11030.000000  Default  29.0000  29.0000
97        car  14324.000000  14324.000000  Default  28.0000  28.0000
108      musk  12244.000000  12244.000000  Default  27.0000  27.0000
133      elon  11679.000000  11679.000000  Default  26.0000  26.0000
..        ...           ...           ...      ...      ...      ...
77       want    746.986183   5036.028295   Topic5  -5.3912   0.0753
136        go    813.448560   8981.731560   Topic5  -5.3060  -0.4181
268      sell    656.835863   4285.099541   Topic5  -5.5198   0.1081
152      look    588.828699   3749.914476   Topic5  -5.6291   0.1323
63      think    579.074865   6583.942150   Topic5  -5.6458  -0.4474

[465 rows x 6 columns], token_table=       Topic      Freq                      Term
term                                            
12108      5  0.983496               accept_doge
12011      1  0.989411  administration_democrats
1111       1  0.070499                    afford
1111       2  0.039811                    afford
1111       3  0.029858                    afford
...      ...       ...                       ...
305        3  0.029914                      year
305        4  0.052641                      year
305        5  0.111691                      year
6952       1  0.981368        yet_unrecognizable
11687      3  0.981622             youtube_video

[1194 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 3, 2, 5, 1])

In [28]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)



In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel = base_model, corpus = corpus, texts = lem_noNone)

In [ ]:
sent_topics_Sorteddf = pd.DataFrame()
sent_topics_Outdf = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, group in sent_topics_Outdf:
    sent_topics_Sorteddf = pd.concat([sent_topics_Sorteddf, 
                                             group.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)
# Reset Index    
sent_topics_Sorteddf.reset_index(drop=True, inplace=True)
# Format
sent_topics_Sorteddf.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

sent_topics_Sorteddf.head()

In [ ]:
# General Code for a basic Radar Chart with only one layer

categories = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4']
dist = 

fig = go.Figure(
    data=[go.Scatterpolar(r=dist, theta=categories, name='Topic Tweet Distribution')],
    layout=go.Layout(
        title=go.layout.Title(text='Restaurant comparison'),
        polar={'radialaxis': {'visible': True}},
        showlegend=False
    )
)

#### ============================================================================

# Below is the grid search the chunky one

### Here
## We 
# Go

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=1500,
                                           passes=8,
                                           alpha=a,
                                           eta=b, 
                                           workers = 7)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=lem_noNone, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 3
max_topics = 9
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [#gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.5)), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['50% Corpus', '75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    results_df = pd.DataFrame(model_results)
    pbar.close()

In [ ]:
results_df = pd.read_csv('Data/first-gs-results-24h')
results_df.drop(columns = 'Unnamed: 0', inplace = True)

In [ ]:
results_df[(results_df['Coherence'] >= 0.38) & (results_df['Topics'] == 5 )]

# =============================================================================

# Below, will be trying to reduce size of corpus to increase model coherence

In [ ]:
len(lem_noNone)

In [ ]:
longer_tweets = [(tweet) for (i, tweet) in enumerate(lem_noNone) if i not in counter]
len(longer_tweets)

In [ ]:
new_id2word = corpora.Dictionary(longer_tweets)
print(len(new_id2word)) 

In [ ]:
new_id2word.filter_extremes(no_below=9, no_above=.90)
print(len(new_id2word))

In [ ]:
new_corpus = [new_id2word.doc2bow(d) for d in longer_tweets]

In [ ]:
# Instantiating a Base LDA model
newer_model = LdaMulticore(corpus=new_corpus,
                          num_topics=5,
                          id2word=new_id2word,
                          workers=6,
                          passes=8,
                          alpha = 0.91 , 
                          eta = 0.61,
                          chunksize=1500,
                          random_state = 100)
newer_words = [re.findall(r'"([^"]*)"',t[1]) for t in newer_model.print_topics()]
newer_topics = [' '.join(t[0:10]) for t in newer_words]

for id, t in enumerate(newer_topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

In [ ]:
newer_perplexity = newer_model.log_perplexity(new_corpus)

# Compute Coherence Score
newer_coherence_model = CoherenceModel(model= newer_model, texts=longer_tweets, 
                                   dictionary=new_id2word, coherence='c_v')

new_coherence_lda_model_base = newer_coherence_model.get_coherence()
print('\nCoherence Score: ', new_coherence_lda_model_base, '\nPerplexity: ', newer_perplexity)

In [ ]:
pyLDAvis.enable_notebook()
gensimvis.prepare(newer_model, new_corpus, new_id2word)

# =============================================================================

# Below --- Need to link tweets to topics, and calculate sentiment by topic

In [ ]:
full

In [ ]:
len(new_corpus)

In [ ]:
# Counter contains perfect indices of all tweets that are short -- directly translates to original full dataframe
counter[:10]

In [ ]:
lem_noNone[19]

In [ ]:
fuck = [1, 24, 3]
for i in enumerate(fuck):
    print(i)

In [ ]:
"""
Note above, 
the original dataframe is indexed from 0, as in it is in the same form as an enumerated list
we created a function earlier to find all of the missing index values somewhere around creating lem_noNone

this function can be used to set proper indexs on the new lists past the funnkiness somehow... easier said than done 
"""

## Attempting to get the top documents per topic 

In [ ]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=newer_model, corpus=new_corpus, texts=longer_tweets)
df_topic_sents_keywords

In [ ]:
df_topic_sents_keywords.Dominant_Topic.value_counts()

In [ ]:
df_topic_sents_keywords.Topic_Keywords.value_counts()

In [ ]:
sent_topics_sorteddf_mallet = pd.DataFrame()
sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=False).head(1)], 
                                            axis=0)
# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)
# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Representative Text"]

sent_topics_sorteddf_mallet.head(10)

In [ ]:
sent_topics_sorteddf_mallet.iloc[2]['Keywords']

In [ ]:
# sent_topics_sorteddf_mallet.iloc[2]['Representative Text']

In [ ]:
# longer_tweets[:10]

In [ ]:
df_topic_sents_keywords

In [ ]:
df_topic_sents_keywords[(df_topic_sents_keywords['Dominant_Topic'] == 4) & (df_topic_sents_keywords['Perc_Contribution'] >= 0.84)]

In [ ]:
len(ind)

In [ ]:
len(counter)

In [ ]:
sum_dropped = counter + ind
len(sum_dropped)

In [ ]:
# full_tester = full.drop(full.index[ind])
full_tester = full.drop(full.index[sum_dropped])
full_tester.reset_index(inplace = True)

In [ ]:
full_tester

In [ ]:
full_tester.tweets.iloc[104646]

In [ ]:
len(longer_tweets)

In [ ]:
len(new_corpus)

In [ ]:
def find_common_strings(string_list, input_string):
    common_strings = []
    for s in string_list:
        if set(s).intersection(set(input_string)) == set(s):
            common_strings.append(s)
    return common_strings

In [ ]:
def find_common_strings(string_list, input_string):
    common_strings = []
    for i, s in enumerate(string_list):
        if set(s).intersection(set(input_string)) == set(s):
            common_strings.append((i, s))
            
    pos_dic = {}
    for i, string in common_strings:
        string_split = string.split(' ')
        input_split = input_string.split(' ')
        common = []
        for word in input_split:
            if word in string_split:
                common.append(word)
        pos_dic[i] = common
        
    
    
    
    return common_strings, pos_dic

In [ ]:
test_string = (' ').join(df_topic_sents_keywords[0].iloc[0])
test_string

In [ ]:
big_list, ind_dict = find_common_strings(full['preprep'], test_string)

In [ ]:
print(len(ind_dict), len(big_list))

In [ ]:
potentials = []
for key, value in ind_dict.items():
    if value != []:
        potentials.append(key)

In [ ]:
for i in potentials:
    print(full['preprep'].iloc[i])

In [ ]:
test_string[::-1].find(' ')

In [ ]:
for s in full['preprep']:
    

In [ ]:
# string_test = find_common_strings(lem_noNone,test_string )
# string_test

# ========================================================================================================================================================================================================================================================================================================================================================================

# SCRAP

# ========================================================================================================================================================================================================================================================================================================================================================================

In [ ]:
for i in range(len(doc_set)):
    print(ldamodel[corpus[i]])

In [ ]:
# CODE WORKS --- JUST USE IT LATER 

# Adding Sentiment
sia = SentimentIntensityAnalyzer()
comp_dic = {}
for i, tweet in full['tweets'].iteritems():
    comp_dic[i] = sia.polarity_scores(tweet)['compound']

comp_scores = pd.Series(comp_dic, name = 'sentiment')
df_sent = full.merge(comp_scores, left_index = True, right_index = True)
df = df_sent.set_index('Unnamed: 0')

In [ ]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df.drop('tokens'))



# CV Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9], 'batch_size':[128, 256]}

# Init Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, cv = 5, param_grid=search_params, n_jobs = -1)



# Grid Search
model.fit(data_vectorized)

# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))


best_lda_model

In [ ]:
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             n_jobs=-2,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)